In [1]:
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt

## Prepare Data

In [2]:
n_data = torch.ones(100, 2)

x0 = torch.normal(2*n_data, 1)
y0 = torch.zeros(100, 1)

x1 = torch.normal(-2*n_data, 1)
y1 = torch.ones(100, 1)

x = torch.cat((x0, x1)).type(torch.FloatTensor)
y = torch.cat((y0, y1)).type(torch.FloatTensor)

## Modeling

In [25]:
# prior modeling
prior1 = (y==1).sum().float() /200.
prior0 = 1 - prior1

In [20]:
# likelihood modeling
mu0 = x[(y==0).squeeze()].mean(0, keepdim=True)
mu1 = x[(y==1).squeeze()].mean(0, keepdim=True)

sigma0 = torch.ones(1,2)
sigma1 = torch.ones(1,2)

In [21]:
K=1
def get_k_likelihoods(X, mu, var):
    if var.eq(0).any():
        raise Exception('variances must be nonzero')
    coeff = 1. / ((2*np.pi * var).prod(dim=1)).sqrt()        # (K )
    sigma_inv = 1./var                                       # (K, features)
    diff = X - mu.unsqueeze(1)                               # (K, examples, feature)    
    exponent = -0.5 * ( diff **2 @ sigma_inv.unsqueeze(2) )
    likelihoods = coeff.view(K,1,1) * torch.exp( exponent )
    return likelihoods.squeeze(2)

## Test

In [22]:
a_sample = torch.ones(1, 2)
x_new = torch.normal(-2*a_sample, 1)  

In [26]:
prob0 = get_k_likelihoods(x_new,mu0,sigma0)
prob1 = get_k_likelihoods(x_new,mu1,sigma1)
(prob1 * prior1) / (prob0 * prior0)  > 1

tensor([[ 1]], dtype=torch.uint8)